En este notebook se revela paso a paso el `funcionamiento interno` de la clase autoscorecard con el dataset del ejemplo 01

<span style='color:blue'>Importamos los módulos

In [1]:
import numpy as np, pandas as pd, pyken as pyk

<span style='color:blue'>Cargamos el dataset separando las variables predictoras (guardadas en X) de la variable objetivo (guardada en y)

In [2]:
from sklearn.datasets import load_breast_cancer

In [3]:
X, y = pd.DataFrame(load_breast_cancer().data, columns=load_breast_cancer().feature_names), load_breast_cancer().target
print('El dataset tiene {} filas y {} columnas (sin incluir el target)'.format(X.shape[0], X.shape[1]))

El dataset tiene 569 filas y 30 columnas (sin incluir el target)


<span style='color:blue'>Echamos un vistazo al dataset

In [4]:
X[X.columns[:10]].head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883


<span style='color:blue'>Como no es buena práctica tener espacios en los nombres de las columnas mejor los sustituyo por guiones bajos

In [5]:
nuevos_nombres = [i.replace(' ', '_') for i in X.columns]
X.columns = nuevos_nombres

<span style='color:blue'>Observamos que todas las variables son de tipo numérico

In [6]:
X.dtypes.unique()

array([dtype('float64')], dtype=object)

<span style='color:blue'>Añadimos una variable ficticia de tipo texto para tener al menos una (a ver si acaba formando parte del modelo)

In [7]:
X['variable_inventada'] = ['a']*(X.shape[0]//3) + ['b']*(X.shape[0]//3) + ['c']*(X.shape[0]//3) + ['d']*(X.shape[0] - 3*(X.shape[0]//3))
print('El dataset tiene {} filas y {} columnas'.format(X.shape[0], X.shape[1]))

El dataset tiene 569 filas y 31 columnas


<span style='color:red'>Hasta aquí el preprocesamiento básico del dataset, que es igual al que hicimos en el notebook del ejemplo 01. Vamos ahora a desengranar como funciona autoscorecard por dentro

<span style='color:blue'>Lo primero que hace autoscorecard es generar una partición train-test. Por defecto hace un 70-30, estratificado en el target y con semilla 123 (para garantizar su replicabilidad por terceros)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify=y)
X_train, X_test = X_train.reset_index(drop=True), X_test.reset_index(drop=True)

<span style='color:blue'>Después, aplica la clase `autogrouping` a todas las variables y guarda los objetos resultantes en un diccionario. Esto genera los buckets automáticos

In [9]:
variables, objetos = X_train.columns, {}
variables_no_agrupadas = []

for variable in variables:
    
    try:
        x = X_train[variable].values
        frenken = pyk.autogrouping(variable).fit(x, y_train)
        objetos[variable] = frenken

    except: variables_no_agrupadas.append(variable)

<span style='color:blue'>También genera un DataFrame oredenado por IV. Además, no considerará a las variables que tengan un IV inverior al umbral mínimo metodológico (0.015, modificable)

In [10]:
tabla_ivs, contador = pd.DataFrame(columns=['variable', 'iv']), 0
for variable in objetos:
    tabla_ivs.loc[contador] = variable, objetos[variable].iv
    contador += 1

tabla_ivs = tabla_ivs.sort_values('iv', ascending=False)
variables_filtroiv = tabla_ivs[tabla_ivs['iv'] >= 0.015]['variable']

In [11]:
variables_def = list(set(variables_filtroiv) - set(variables_no_agrupadas))

<span style='color:blue'>Podemos ver cuales son las variables con mayor IV (= information value). Podríamos probar a generar una scorecard seleccionando las n primeras de mayor IV

In [12]:
tabla_ivs.head()

,variable,iv
27,worst_concave_points,6.377094
20,worst_radius,6.169409
22,worst_perimeter,6.148091
23,worst_area,5.854883
7,mean_concave_points,5.754284


<span style='color:red'>Por ejemplo, veamos como se haría la scorecard generada con las tres primeras variables por IV: `worst_concave_points`, `worst_radius`, `worst_perimeter`

<span style='color:blue'>En *features* pondríamos las variables que queremos que formen parte de la scorecard, en este caso las tres que hemos dicho

In [13]:
features = ['worst_concave_points', 'worst_radius', 'worst_perimeter']

<span style='color:blue'>Si se quisieran usar agrupaciones manuales habría que ponerlas dentro del diccionario *user_breakpoints*, por ahora lo dejamos vacío

In [14]:
user_breakpoints = {}

<span style='color:blue'>En *final_breakpoints* se actualizarían los grupos en caso de haber introducido agrupaciones manuales en user_breakpoints. Ahora se utilizaran las agrupaciones automáticas

In [15]:
final_breakpoints = pyk.compute_final_breakpoints(features, objetos, user_breakpoints)

<span style='color:blue'>Antes de calcular el modelo, la clase autoscorecard aplica un *tratamiento* a las columnas del dataframe

In [16]:
info = pyk.compute_info(X_train, features, final_breakpoints)

In [17]:
df_train_adapted = pyk.adapt_data(X_train, y_train, features, final_breakpoints)

<span style='color:blue'>Calculamos ya la scorecard (pero solo como *tarjeta de puntuación* en un pd.DataFrame, el objeto en sí se obtiene usando clase autoscorecard)

In [18]:
scorecard, features_length = pyk.compute_scorecard(df_train_adapted, features, info)

<span style='color:blue'>Podemos aplicar esta tarjeta de puntuación al data de entrenamiento y ver las *métricas asociadas*

In [19]:
df_train_adapted_scored, ks_train, gini_train = pyk.apply_scorecard(df_train_adapted, scorecard, info, metrics=['ks', 'gini'], print_log=True)

El modelo tiene un 89.82% de KS y un 97.66% de Gini en esta muestra


<span style='color:blue'>Hacemos lo mismo con los datos del test (el data de validación del 30%, habitualmente también llamado *hold out*)

In [20]:
df_test_adapted = pyk.adapt_data(X_test, y_test, features, final_breakpoints)
df_test_adapted_scored, ks_test, gini_test = pyk.apply_scorecard(df_test_adapted, scorecard, info, metrics=['ks', 'gini'], print_log=True)

El modelo tiene un 92.20% de KS y un 97.63% de Gini en esta muestra


<span style='color:blue'>No nos ha salido un mal modelo, también porque es que este dataset es MUY de juguete... La clase autoscorecard hace todo igual *salvo la selección de variables*...
    
<span style='color:blue'>Es decir, *no elige las variables en función de su IV sino que se utilizan cualquiera de las siguientes dos aproximaciones*:
    
*Método `forward` con métrica de `ks` o `gini`*: En el primer paso se parte del modelo vacío, es decir el modelo que no tiene ninguna variable. Entonces para cada variable candidata (aquellas agruapdas con un IV >= 0.015) se genera un modelo que tienen a esa variable como única variable del modelo, así de entre todos estos modelos univariable se mira cual sería el que da un mayor valor de la métrica (ks o gini) en el data del train. Aquella variable cuyo modelo de el MÁXIMO valor de la métrica se selecciona en este primer paso.
    
En el segundo paso se parte del conjunto que ya tiene la primera variable seleccionada del paso anterior. Ahora, se consideran TODAS las demás variables candidatas para generar TODOS los modelos de dos variables distintos posibles donde la primera es la que ya habíamos seleccionado en el paso anterior. Bien, pues de entre todos estos modelos 2-variables aquel que de el MÁXIMO valor de la métrica en el train nos indica cual es la segunda variable que seleccionamos: la que se ha añadido a la que ya teníamos para generar este modelo.
        
Note el lector que si el número de variables candidatas es n, entonces en el paso 1 se consideran n modelos 1-variable y en el paso 2 se considerarían (n-1) modelos 2-variables. Sin embargo, estos modelos 2-variables son más costosos desde el punto de vista computacional por tener una variable más y esto hace que los tiempos vayan aumentando en cada paso aunque el número de modelos que se consideran sea cada vez menor.
        
El proceso se detiene cuando se alcanza el máximo número de pasos permitidos o cuando la métrica no mejora más de un umbral (0.40 para KS y 0.25 para Gini) la del paso anterior.
    
*Método `stepwise` con métrica de `p-valor` (configuración por defecto de la clase autoscorecard)*: Se realiza un forward como el anteriormente descrito pero con algunas modificaciones. Se empieza buscando la variable que genera el modelo 1-variable en donde esta variable tiene el p-valor más bajo (los p-valores se calculan a nivel variable, no a nivel modelo). En un segundo paso se consideran todos los modelos 2-variables donde la primera variable es la elegida en el paso anterior y la segunda es cualquiera de las candidatas restantes. Bien, pues en cada uno de estos modelos se CALCULAN los p-valores de las dos variables y se elige el que tenga el p-valor de la variable que se está probando más bajo.
    
La gracia ahora es que aquí en cada paso se RECALCULAN los p-valores de TODAS las variables involucradas en el modelo de turno, no solo el de la variable candidata a entrar si no también los p-valores del resto de variables ya seleccionadas de forma que si en algún momento alguno de ellos es superior a 0.01 (nivel de significancia metodológico) entonces esta variable SALE del modelo. Esto ocurrirá cuando la variable que acaba de entrar está fuertemente correlada con la que está saliendo pero por algún motivo la que entra se combina mejor con el resto de variables del modelo y tiene una aportación 'mayor' al modelo (estadísticamente hablando).
    
El proceso se detiene cuando se alcanza el máximo número de pasos permitidos o cuando ninguna de las variables retadas tiene un p-valor inferior a 0.01.

<span style='color:blue'>Hemos implementado la función `feature_selection` que replica este proceso, la usamos con sus valores por defecto para obtener el mismo set de variables que en el ejemplo 01

In [21]:
final_breakpoints = pyk.compute_final_breakpoints(variables_def, objetos, user_breakpoints)
info = pyk.compute_info(X_train, variables_def, final_breakpoints)
df_train_adapted = pyk.adapt_data(X_train, y_train, variables_def, final_breakpoints)
df_test_adapted = pyk.adapt_data(X_test, y_test, variables_def, final_breakpoints)

In [22]:
features = pyk.features_selection(df_train_adapted, [], variables_def, info, muestra_test=df_test_adapted)

Step 01 | Time - 0:00:00.743492 | p-value = 4.93e-32 | Gini train = 83.97% | Gini test = 87.30% ---> Feature selected: mean_concavity
Step 02 | Time - 0:00:00.946147 | p-value = 1.38e-14 | Gini train = 96.82% | Gini test = 97.24% ---> Feature selected: worst_perimeter
Step 03 | Time - 0:00:01.132104 | p-value = 4.31e-06 | Gini train = 98.34% | Gini test = 98.07% ---> Feature selected: worst_texture
Step 04 | Time - 0:00:01.231529 | p-value = 5.11e-04 | Gini train = 98.92% | Gini test = 97.06% ---> Feature selected: worst_smoothness
Step 05 | Time - 0:00:01.383086 | p-value = 1.62e-03 | Gini train = 99.34% | Gini test = 98.51% ---> Feature selected: radius_error
Step 05 | Time - 0:00:00.000000 | p-value = 1.54e-02 | Gini train = 99.25% | Gini test = 98.22% ---> Feature deleted : mean_concavity
Step 06 | Time - 0:00:01.512500 | p-value = 2.28e-03 | Gini train = 99.60% | Gini test = 98.77% ---> Feature selected: worst_concavity
-------------------------------------------------------------

<span style='color:blue'>Ahora ya sí la scorecard sale igual que ene ejemplo 01

In [23]:
scorecard, features_length = pyk.compute_scorecard(df_train_adapted, features, info)
df_train_adapted_scored, ks_train, gini_train = pyk.apply_scorecard(df_train_adapted, scorecard, info, metrics=['ks', 'gini'], print_log=True)
df_test_adapted_scored, ks_test, gini_test = pyk.apply_scorecard(df_test_adapted, scorecard, info, metrics=['ks', 'gini'], print_log=True)

El modelo tiene un 95.55% de KS y un 99.60% de Gini en esta muestra
El modelo tiene un 95.63% de KS y un 98.77% de Gini en esta muestra
